
## Project:  Allstate Claims Severity
#### Author:   Joshep Downs, James Peng, Megan Pera, Diana Rodenberger 
#### Purpose:  Predicting cost and severity of claims for AllState
#### Created:  10/29/2016

In [ ]:
import unittest

# General libraries.
import re, os, sys
import numpy as np
import pandas as pd
import logging
import matplotlib.pyplot as plt

from sklearn.pipeline import Pipeline
#from sklearn.feature_extraction import 
from sklearn import preprocessing

from sklearn.utils import shuffle

from sklearn.metrics import mean_absolute_error

from sklearn import linear_model

from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction import DictVectorizer as DV

from sklearn.ensemble import ExtraTreesRegressor
from sklearn.ensemble import RandomForestRegressor

from sklearn.model_selection import KFold

import xgboost as xgb

from sklearn.feature_extraction import DictVectorizer as DV

In [9]:
TARGET = 'loss'
TARGET_LOG = 'log_loss'
ID = 'id'

DATA_IN_DIR = "./data_in"
DATA_OUT_DIR = "./data_out"

TRAIN_CSV = "{0}/train.csv".format(DATA_IN_DIR)
TEST_CSV = "{0}/test.csv".format(DATA_IN_DIR)
SUBMISSION_CSV = "{0}/submission.csv".format(DATA_OUT_DIR)

print(TRAIN_CSV, TEST_CSV,SUBMISSION_CSV)

train_data = pd.read_csv(TRAIN_CSV)
test_data = pd.read_csv(TEST_CSV)

print(train_data.shape)
print(test_data.shape)


./data_in/train.csv ./data_in/test.csv ./data_out/submission.csv
(188318, 132)
(125546, 131)


In [10]:
test_data[TARGET] = 0
test_data[TARGET_LOG] = np.log10(train_data[TARGET]+1)

train_data['train'] = 1
test_data['train'] = 0

nrows_train = train_data.shape[0]
nrows_test = test_data.shape[0]
print(nrows_train,nrows_test )

train_test = pd.concat((train_data, test_data), axis=0).reset_index(drop=True)

features = train_test.columns
cats = [feat for feat in features if 'cat' in feat]
conts = [feat for feat in features if 'cont' in feat]

print(train_test.shape)

188318 125546
(313864, 134)


In [ ]:
train_test[cats]

In [6]:
cats_train_test = pd.get_dummies(train_test[cats])
train_test2 = pd.concat((cats_train_test, train_test[conts] ), axis=1).reset_index(drop=True)


X = train_test2[:nrows_train]
y = train_test[TARGET_LOG][:nrows_train]
Id = train_test[ID][:nrows_train]

X_train, X_dev, y_train, y_dev, id_train, id_dev = train_test_split( X, y, Id, test_size=0.33, random_state=1)


In [5]:
print(train_test2.shape)
train_test2

(313864, 1190)


,cat1_A,cat1_B,cat10_A,cat10_B,cat100_A,cat100_B,cat100_C,cat100_D,cat100_E,cat100_F,...,cont13,cont14,cont2,cont3,cont4,cont5,cont6,cont7,cont8,cont9
0,1,0,1,0,0,1,0,0,0,0,...,0.822493,0.714843,0.245921,0.187583,0.789639,0.310061,0.718367,0.335060,0.30260,0.67135
1,1,0,0,1,0,0,0,0,0,0,...,0.611431,0.304496,0.737068,0.592681,0.614134,0.885834,0.438917,0.436585,0.60087,0.35127
2,1,0,0,1,0,0,0,0,0,0,...,0.195709,0.774425,0.358319,0.484196,0.236924,0.397069,0.289648,0.315545,0.27320,0.26076
3,0,1,1,0,0,0,0,0,0,0,...,0.605077,0.602642,0.555782,0.527991,0.373816,0.422268,0.440945,0.391128,0.31796,0.32128
4,1,0,0,1,0,0,0,0,0,1,...,0.246011,0.432606,0.159990,0.527991,0.473202,0.704268,0.178193,0.247408,0.24564,0.22089
5,1,0,1,0,0,0,0,0,0,0,...,0.345247,0.726792,0.681761,0.634224,0.373816,0.302678,0.364464,0.401162,0.26847,0.46226
6,1,0,1,0,0,0,0,0,0,0,...,0.342239,0.382931,0.737068,0.613660,0.189137,0.295397,0.381515,0.363768,0.24564,0.40455
7,1,0,1,0,0,0,0,0,0,0,...,0.859764,0.242416,0.488789,0.263570,0.623770,0.473767,0.867021,0.583389,0.90267,0.84847
8,1,0,0,1,0,0,1,0,0,0,...,0.756454,0.361191,0.555782,0.440642,0.473202,0.281143,0.628534,0.384099,0.61229,0.38249
9,1,0,1,0,0,0,0,0,0,1,...,0.822493,0.294523,0.358319,0.356819,0.802892,0.310061,0.713343,0.469223,0.30260,0.67135


In [7]:
print('X_train shape: ',X_train.shape)
print('y_train shape: ',y_train.shape)
print('X_dev shape: ',X_dev.shape)
print('y_dev shape: ',y_dev.shape)

X_train shape:  (126173, 1190)
y_train shape:  (126173,)
X_dev shape:  (62145, 1190)
y_dev shape:  (62145,)


In [ ]:
df_data_encoded = df_data.copy()

# encoding into the categorical value
le = preprocessing.LabelEncoder()
for c in df_data_encoded.columns:
    if c.find('cat') >=0: # -1: substring not found, >=0, starting index
        df_data_encoded[c] = le.fit_transform(df_data_encoded[c])

np.random.seed(100)


col = list(df_data_encoded.columns)
col.remove('loss')
col.remove('id')

X = df_data_encoded[col]
y = np.log10(df_data_encoded.loss)
id = df_data_encoded.id

X_train, X_dev, y_train, y_dev, id_train, id_dev = train_test_split( X, y, id, test_size=0.33, random_state=1)

In [8]:
#id_train, x_train,  y_train = shuffle( df_train.id, df_train[col] , df_train.loss, random_state=0)

pd.options.mode.chained_assignment = None


lr = linear_model.LinearRegression()

# Train the model using the training sets
lr.fit(X_train, y_train)

y_pred = lr.predict(X_dev)

s = cross_val_score(lr, X_train, y_train, scoring='neg_mean_absolute_error')

print('mean_absolute_error on training data: {0}'.format(s))

mae = mean_absolute_error(y_dev, y_pred)
print('mean_absolute_error on test data {0}'.format(mae))

print('end')
#if __name__ == '__main__':
    #unittest.main()


ValueError: Input contains NaN, infinity or a value too large for dtype('float64').

In [11]:
# fitting model on training data
xgbr = xgb.XGBRegressor(max_depth=6, n_estimators=500, learning_rate=0.1, subsample=0.8, colsample_bytree=0.4,
                     min_child_weight = 3,  seed=7)
xgbr.fit(X_train, y_train)

print(xgbr)
#Making predictions
y_pred = xgbr.predict(X_dev) 

s = cross_val_score(xgbr, X_train, y_train, scoring='neg_mean_absolute_error')

print('mean_absolute_error on training data: {0}'.format(s))

mae = mean_absolute_error(y_dev, y_pred)
print('mean_absolute_error on test data {0}'.format(mae))


XGBRegressor(base_score=0.5, colsample_bylevel=1, colsample_bytree=0.4,
       gamma=0, learning_rate=0.1, max_delta_step=0, max_depth=6,
       min_child_weight=3, missing=None, n_estimators=500, nthread=-1,
       objective='reg:linear', reg_alpha=0, reg_lambda=1,
       scale_pos_weight=1, seed=7, silent=True, subsample=0.8)


ValueError: Input contains NaN, infinity or a value too large for dtype('float64').

In [ ]:
df_test = pd.read_csv('./data_in/test.csv')

In [ ]:
df_test_encoded = df_test.copy()

# encoding into the categorical value
le = preprocessing.LabelEncoder()
for c in df_test_encoded.columns:
    if c.find('cat') >=0: # -1: substring not found, >=0, starting index
        df_test_encoded[c] = le.fit_transform(df_test_encoded[c])

np.random.seed(100)


col = list(df_test_encoded.columns)
col.remove('id')

X_test = df_test_encoded[col]
id_test = df_test_encoded.id

In [ ]:
#id_train, x_train,  y_train = shuffle( df_train.id, df_train[col] , df_train.loss, random_state=0)

pd.options.mode.chained_assignment = None

#use same linear model previously fit with training data
y_test_log_pred = lr.predict(X_test)

# convert from log10 scale to linear scale 
y_test_pred = np.power(10, y_test_log_pred)

# assamble the final dataset
y_test_predicted_loss=pd.DataFrame(id_test)

y_test_pred_df=pd.DataFrame({"loss": y_test_pred})

y_test_predicted_loss=pd.concat([y_test_predicted_loss,y_test_pred_df], axis=1)


#check final output
print("number of ids: ", len(id_test))
print("number of rows in predicted test set: ",len(y_test_predicted_loss) )

#create csv file to submit
y_test_predicted_loss.to_csv('./data_out/AllState_LossPrediction_v1.csv', encoding='utf-8', index=False)
